In [7]:
class TrafficDataAnalysis:
    def __init__(self, traffic_data, crash_data):
        self.traffic_data = traffic_data
        self.crash_data = crash_data
        self.traffic_volume = self._build_traffic_volume()
        self.target_accident_types = [
            "Collision with vehicle",
            "Collision with a fixed object",
            "Vehicle overturned (no collision)",
            "collision with some other object",
            "Struck Pedestrian",
            "No collision and no object struck",
            "Struck animal",
            "Fall from or in moving vehicle",
            "Other accident"
        ]
        self.collision_counts = {acc_type: {} for acc_type in self.target_accident_types}
        self.counted_accidents = set()

    def _build_traffic_volume(self):
        """Extract traffic volume data from traffic data."""
        traffic_volume = {}
        for road in self.traffic_data:
            object_id = road['_source']['OBJECTID']
            all_vehs_mmw = road['_source']['ALLVEHS_MMW']
            traffic_volume[object_id] = all_vehs_mmw
        return traffic_volume

    def _get_min_max_coordinates(self, coords):
        """Calculate the minimum and maximum coordinates."""
        min_lon = min(point[0] for point in coords)
        max_lon = max(point[0] for point in coords)
        min_lat = min(point[1] for point in coords)
        max_lat = max(point[1] for point in coords)
        return min_lon, max_lon, min_lat, max_lat

    def _is_within_bounds(self, location, min_lon, max_lon, min_lat, max_lat, tolerance=0.02):
        """Check if a location is within the given boundaries with a tolerance."""
        return (min_lon - tolerance) <= location['lon'] <= (max_lon + tolerance) and \
               (min_lat - tolerance) <= location['lat'] <= (max_lat + tolerance)

    def analyze_accidents(self):
        """Analyze accidents to compute collision counts based on traffic data."""
        for road in self.traffic_data:
            object_id = road['_source']['OBJECTID']
            coords = road['_source']['coordinates']['coordinates']
            min_lon, max_lon, min_lat, max_lat = self._get_min_max_coordinates(coords)

            for accident in self.crash_data:
                accident_id = accident['_id']
                if accident_id not in self.counted_accidents:
                    location = {'lon': accident['_source']['Location']['lon'], 'lat': accident['_source']['Location']['lat']}
                    accident_type = accident['_source']['ACCIDENT_TYPE']
                    if accident_type in self.target_accident_types and self._is_within_bounds(location, min_lon, max_lon, min_lat, max_lat):
                        if object_id not in self.collision_counts[accident_type]:
                            self.collision_counts[accident_type][object_id] = 0
                        self.collision_counts[accident_type][object_id] += 1
                        self.counted_accidents.add(accident_id)

    def plot_collisions(self):
        """Plot the relationship between traffic volumes and number of collisions."""
        import matplotlib.pyplot as plt
        for acc_type in self.target_accident_types:
            volumes = []
            collisions = []
            for object_id in self.collision_counts[acc_type]:
                if object_id in self.traffic_volume:
                    volumes.append(self.traffic_volume[object_id])
                    collisions.append(self.collision_counts[acc_type][object_id])

            if volumes and collisions:
                plt.figure(figsize=(10, 6))
                plt.scatter(volumes, collisions, alpha=0.5)
                plt.title(f'Relationship between Traffic Volume and Number of Collisions for {acc_type}')
                plt.xlabel('Traffic Volume')
                plt.ylabel('Number of Collisions')
                plt.grid(True)
                plt.show()
            else:
                print(f"No data to plot for {acc_type}")

# Usage example:
# analysis = TrafficDataAnalysis(all_traffic_data, all_crash_data)
# analysis.analyze_accidents()
# analysis.plot_collisions()
